In [68]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import string
import datetime
import re
%load_ext autoreload
%autoreload 2

#modeling imports
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
plt.style.use('fivethirtyeight')

#tf imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score


#multinomial nb
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from scipy.sparse.linalg import svds
from preprocc import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PREPROCESSING

In [ ]:
X_train, y_train, X_test, y_test = load_data()
X_train, X_test = preprocess(X_train, X_test)

# Modeling

In [95]:
from preprocc import *
train_data, test_data = tokenize(X_train, X_test)
model = eight_way(train_data, test_data, y_train, y_test, batch_size=20)


221131 first!!!


In [79]:
#score on the test set! 
score = model.evaluate(test_data, y_test)

NameError: name 'model' is not defined

In [139]:
print("toxic label %s: %.2f%%" % (model.metrics_names[1], score[1]*100))

toxic label acc: 91.06%


In [148]:
#train models for all the other labels 
y_train
labels = y_train.columns
labels


In [149]:
for label in labels:
    print("====================================")
    print("====================================")
    print("starting fit on", label)
    print("====================================")
    print("====================================")
    if label == 'toxic':
        continue #already trained toxic
    
    #just reset the model, bc idk how to make sure it deletes old fit
    model = Sequential()
    #first layer is embedding, takes in size of vocab, 100 dim embedding, and 150 which is length of the comment 
    model.add(Embedding(num_words, 100, input_length=150)) 
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #fit the data to the next label 
    model.fit(data, np.array(y_train[label]), validation_split=.3, epochs=3)
    model_json = model.to_json()
    
    name_json = label + '.json'
    name_h5 = label + '.h5'
    
    #save into json file
    with open(name_json, "w") as json_file:
        json_file.write(model_json)
    
    # serialize weights to HDF5
    model.save_weights(name_h5)
    print("Saved", label,"to disk")
    
    #lastly, evaluate on test 
    score = model.evaluate(test_data, y_test[label])
    print(label,"%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    print("====================================")
    print("====================================")
    print("done with", label,"starting next fit")
    print("====================================")
    print("====================================")
    

starting fit on toxic
starting fit on severe_toxic
Train on 111699 samples, validate on 47872 samples
Epoch 1/3
111699/111699 [==============================] - 1719s 15ms/step - loss: 0.0356 - acc: 0.9901 - val_loss: 0.0246 - val_acc: 0.9904
Epoch 2/3
111699/111699 [==============================] - 1850s 17ms/step - loss: 0.0205 - acc: 0.9915 - val_loss: 0.0255 - val_acc: 0.9908
Epoch 3/3
111699/111699 [==============================] - 2209s 20ms/step - loss: 0.0147 - acc: 0.9936 - val_loss: 0.0293 - val_acc: 0.9900
Saved severe_toxic to disk
63978/63978 [==============================] - 81s 1ms/step
severe_toxic acc: 98.44%
done with severe_toxic starting next fit
starting fit on obscene
Train on 111699 samples, validate on 47872 samples
Epoch 1/3
111699/111699 [==============================] - 2180s 20ms/step - loss: 0.0821 - acc: 0.9750 - val_loss: 0.0540 - val_acc: 0.9806
Epoch 2/3
111699/111699 [==============================] - 1891s 17ms/step - loss: 0.0431 - acc: 0.9841 - 

In [ ]:

#try # 2 dont use 
X_train['seq'] = tokenizer.texts_to_sequences(X_train['comment_text'])
X_test['seq'] = tokenizer.texts_to_sequences(X_test['comment_text'])

In [102]:
def pad_shorter(arr, max_len):
    for n in range(len(arr), max_len):
        arr.append(None) 
    return arr

max_len = 100
print("min length before:", X_train.seq.map(lambda x: len(x)).min())
X_train['seq'] = X_train['seq'].apply(lambda x: pad_shorter(x, max_len))

min length before: 1


In [103]:
print("min length after:", X_train.seq.map(lambda x: len(x)).min())
X_train

min length after: 100


,comment_text,seq
0,Explanation\nWhy the edits made under my usern...,"[689, 76, 2, 127, 131, 178, 30, 673, 4512, 120..."
1,D'aww! He matches this background colour I'm s...,"[96146, 53, 2636, 14, 556, 3810, 74, 4557, 270..."
2,"Hey man, I'm really not trying to edit war. It...","[413, 438, 74, 135, 15, 250, 3, 72, 315, 79, 5..."
3,"""\nMore\nI can't make any real suggestions on ...","[58, 8, 229, 98, 55, 329, 1437, 16, 2134, 8, 6..."
4,"You, sir, are my hero. Any chance you remember...","[7, 1678, 20, 30, 3517, 55, 1070, 7, 580, 40, ..."
...,...,...
159566,""":::::And for the second time of asking, when ...","[5, 13, 2, 428, 85, 4, 903, 83, 21, 314, 563, ..."
159567,You should be ashamed of yourself \n\nThat is ...,"[7, 57, 17, 4653, 4, 207, 10, 9, 6, 3328, 232,..."
159568,"Spitzer \n\nUmm, theres no actual article for ...","[34279, 7331, 5209, 47, 738, 24, 13, 8168, 351..."
159569,And it looks like it was actually you who put ...,"[5, 12, 575, 50, 12, 25, 211, 7, 63, 202, 16, ..."


In [105]:
def trunc_longer(arr, max_len):
    if len(arr) > max_len:
        arr = arr[:max_len]
    return arr

max_len = 100
print("max length before:", X_train.seq.map(lambda x: len(x)).max())
X_train['seq'] = X_train['seq'].apply(lambda x: trunc_longer(x, max_len))
print("max length after:", X_train.seq.map(lambda x: len(x)).max())
X_train.head()

#here all the comments should be the same size. go back and check what the mean/median 
#length of comment (by word) is, 100 might be a hyperparameter we should tune. 
#could also be checkign what the lengths are of toxic comments vs. normal comments. 

max length before: 1403
max length after: 100


,comment_text,seq
0,Explanation\nWhy the edits made under my usern...,"[689, 76, 2, 127, 131, 178, 30, 673, 4512, 120..."
1,D'aww! He matches this background colour I'm s...,"[96146, 53, 2636, 14, 556, 3810, 74, 4557, 270..."
2,"Hey man, I'm really not trying to edit war. It...","[413, 438, 74, 135, 15, 250, 3, 72, 315, 79, 5..."
3,"""\nMore\nI can't make any real suggestions on ...","[58, 8, 229, 98, 55, 329, 1437, 16, 2134, 8, 6..."
4,"You, sir, are my hero. Any chance you remember...","[7, 1678, 20, 30, 3517, 55, 1070, 7, 580, 40, ..."


In [ ]:
#now, start the RNN model somehow 
sequences = tokenizer.texts_to_sequences

In [2]:
sequences

NameError: name 'sequences' is not defined

In [17]:
#load the pandas datasets into a tensor
# train = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
# test = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))


general notes:

i feel like we should train the length of comment as a hyperparameter. need to figure out how long most comments are and set to that, not arbitrarily 150. find most common length, medial length, etc. worried that the current model is cutting off any hashtags at the end, etc. 

In [1]:
!which python 

/Library/Frameworks/Python.framework/Versions/2.7/bin/python
